<a href="https://colab.research.google.com/github/31indianaSKKU/SIS_DL_AML/blob/main/AML_Lightgbm_teamproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

df_raw = pd.read_excel('./drive/MyDrive/MINS_trainset.xlsx')
print(df_raw.shape)

(99, 34)


In [ ]:
df_raw['AMI'].value_counts()  # imbalanced! (9:1)

0    89
1    10
Name: AMI, dtype: int64

In [ ]:
X = df_raw.iloc[:, :-1]
y = df_raw.iloc[:, -1]

In [ ]:
from sklearn.preprocessing import LabelEncoder
enc_dept = LabelEncoder().fit(X['dept'])
X['dept'] = enc_dept.transform(X['dept'])
enc_sex = LabelEncoder().fit(X['sex'])
X['sex'] = enc_sex.transform(X['sex'])
print(X['dept'].value_counts().to_dict())
print(X['sex'].value_counts().to_dict())

{0: 73, 1: 22, 4: 2, 3: 1, 2: 1}
{1: 59, 0: 40}


In [ ]:
# sklearn fold를 써서 최적의 parameter를 구함
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=2023).split(X=X, y=y)

param_grid = {
    'max_depth': [-1, 3, 5, 7],
    'learning_rate': [0.005, 0.01, 0.05],
    'min_child_samples': [1, 3, 5],
}

# target = 0/1인 binary classification에 맞게.
lgbc = lgb.LGBMClassifier(boosting_type='gbdt', objective='binary', seed=2023,
                          metric=['binary_logloss', 'auc'], verbose=2)

cv = GridSearchCV(estimator=lgbc, param_grid=param_grid, cv=skf, verbose=2)
lgb_model = cv.fit(X=X, y=y)

# print(lgb_model.best_params_, lgb_model.best_score_)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 2 and depth = 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 2 and depth = 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 2 and depth = 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 2 and depth = 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 2 and depth = 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 2 and depth = 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 2 and d

In [ ]:
# gridSearch를 이용해 찾은 최적의 모델/파라미터 확인

bst = lgb_model.best_estimator_
params = bst.get_params()

# 파라미터 값 확인
print(params)

# 원하는 파라미터 값 변경
params['max_depth'] = 10
params['learning_rate'] = 0.001
params['n_estimators'] = 130


# 변경된 파라미터로 모델 재학습
bst.set_params(**params)




{'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0, 'importance_type': 'split', 'learning_rate': 0.001, 'max_depth': 10, 'min_child_samples': 3, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 180, 'n_jobs': -1, 'num_leaves': 31, 'objective': 'binary', 'random_state': None, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'silent': 'warn', 'subsample': 1.0, 'subsample_for_bin': 200000, 'subsample_freq': 0, 'seed': 2023, 'metric': ['binary_logloss', 'auc'], 'verbose': 2}


LGBMClassifier(learning_rate=0.001, max_depth=10,
               metric=['binary_logloss', 'auc'], min_child_samples=3,
               n_estimators=130, objective='binary', seed=2023, verbose=2)

In [ ]:
# categorical feature인 'dept'가, 몇 번째 변수인지 index 확인
cate_cols = ['dept', 'sex', 'op_em', 'op_vascular', 'CAD', 'PAD', 'HF', 'AF']
col_idx_mapping = {c:i for i, c in enumerate(X.columns)}
list_indices = []
for c in cate_cols :
    list_indices.append(col_idx_mapping[c])
    X[c] = X[c].astype('category')
print(list_indices)

# lgbm dataset 생성 w/categorical_feature
train_data = lgb.Dataset(X, label=y, categorical_feature = list_indices)

[0, 2, 17, 18, 8, 9, 11, 12]


In [ ]:
#  파라미터로 학습
best_params = bst.get_params()
best_lgbc = lgb.train(best_params, train_data, categorical_feature=list_indices, valid_sets=[train_data])

[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: importance_type
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: importance_type
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 10, number of negative: 89
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.874459
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.498208
[LightGBM] [Debug] init for col-wise cost 0.000411 seconds, init for row-wise cost 0.000455 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000437 seconds.
You can set `force_row_wi

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/usr/local/lib/python3.10/dist-packages/lightgbm/basic.py:1487: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  _log_warning(f'{key} keyword has been found in `params` and will be ignored.\n'


In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
y_pred_proba = best_lgbc.predict(X)
y_pred = (y_pred_proba>0.5)*1
confusion_matrix(y, y_pred, labels=[0,1])

array([[89,  0],
       [10,  0]])

In [ ]:
pd.DataFrame([(nm, imp) for nm, imp in zip(best_lgbc.feature_name(), best_lgbc.feature_importance()) if imp > 0.], columns=['feature_name', 'importance'])\
    .sort_values(by='importance', ascending=False)

,feature_name,importance
17,time,333
3,ht,251
4,wt,250
1,age,248
20,duration_a,164
21,mean_MAP,126
14,Cr,120
5,DM,109
10,kidney_dis,106
25,HR,100


In [ ]:
df_test = pd.read_excel('./drive/MyDrive/MINS_testset.xlsx')
print(df_test.shape)

(67, 34)


In [ ]:
# string to category
df_test['dept'] = enc_dept.transform(df_test['dept'].replace('ENT', 'GS'))   # 학습데이터에 존재하지 않았던 category는, 최다 category로 임의 변경함.
df_test['sex'] = enc_sex.transform(df_test['sex'])

In [ ]:
# 예측데이터에 대 lgb.Dataset 생성
for c in cate_cols :
    df_test[c] = df_test[c].astype('category')
test_data = lgb.Dataset(df_test.drop(['AMI'], axis=1), label=df_test['AMI'], categorical_feature = cate_cols)

In [ ]:
y_test_pred = best_lgbc.predict(df_test.drop(['AMI'], axis=1))
y_test_pred = (y_test_pred>0.12)*1
from sklearn.metrics import classification_report
print(classification_report(df_test['AMI'], y_test_pred, target_names=['0','1']))
confusion_matrix(df_test['AMI'], y_test_pred, labels=[0,1])

              precision    recall  f1-score   support

           0       0.96      0.87      0.91        60
           1       0.38      0.71      0.50         7

    accuracy                           0.85        67
   macro avg       0.67      0.79      0.71        67
weighted avg       0.90      0.85      0.87        67



array([[52,  8],
       [ 2,  5]])